In [ ]:
# Standard imports
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import plotly.offline as po
import plotly

## Stoßdaten Güterwagen
Die DB Cargo AG hat Ihre Güterwagen mit modernen Telematikgeräten ausgerüstet. Verschiedene Sensoren liefern mit regelmäßiger Frequenz und auch eventbasiert Daten.

In diesem Datensatz sind Informationen zu größeren Stößen, gemessen durch einen Beschleunigungssensor am Wagen enthalten.

- wagonID = Eindeutige ID des Güterwagens
- latitude = Geographische Breite
- longitude = Geographische Länge
- speed = GNSS Geschwindigkeit des Wagens in km/h
- delta_timestamp = Messzeitpunkt relativ zu t0
- shock_duration = Dauer des Shockereignisses in ms
- shock_x_axis = Rohdaten für die x-Achse in g mit 500Hz
- x_axis = Max. Amplitude für x-Achse in g
- shock_y_axis = Rohdaten für y-Achse in g mit 500Hz
- y_axis = Max. Amplitude für y-Achse in g
- shock_z_axis = Rohdaten für z-Achse in g mit 500Hz
- z_axis = Max. Amplitude für z-Achse in g

[Link zur Mobilithek](https://mobilithek.info/offers/573487566471229440)

In [ ]:
# Load data - Attention: 1.8 GB!
# Online data - Mobilithek doesn't allow download via pandas
!wget https://mobilithek.info/mdp-api/files/aux/573487566471229440/ShockData.csv
# Local data
df = pd.read_csv('ShockData.csv')
df.head()

### Methode zur Darstellung von Fourier-Transformierten

In [ ]:
def fftplot(x, ax, label, T, log = True, window = True):
    X = np.abs(np.fft.fft(x))
    n = len(X)
    if window:
        w = np.hamming(n)
        X = np.multiply(X,w)
    F = np.fft.fftfreq(n, d=T)
    if log:
        ax.semilogy(F[0:int(np.floor(n/2))], X[0:int(np.floor(n/2))], label = label)
    else:
        ax.plot(F[0:int(np.floor(n/2))], X[0:int(np.floor(n/2))], label = label)
    ax.grid()

### Übersichtsplot eines Stoßes in drei Achsen und den Fourier-Transformierten

In [ ]:
ax = plt.figure(figsize = (10,10))
ind = df[df['shock_duration'] >= 910].index
for i in ind:
    str1 = df.iloc[i]['shock_x_axis']
    if isinstance(str1, str):
        l = str1.split(',')
        y = list(map(float,l[0:-2]))
        plt.subplot(3,2,1)
        plt.plot(y, label = '$x$')
        plt.legend()
        # FFT transformation of variable
        ax2 = plt.subplot(3,2,2)
        fftplot(y, ax2, '$\mathcal{F}(x)$', 1/500, log = True)
        plt.legend()
plt.subplot(3,2,3)
for i in ind:
    str1 = df.iloc[i]['shock_y_axis']
    if isinstance(str1, str):
        l = str1.split(',')
        y = list(map(float,l[0:-2]))
        plt.plot(y, label = '$y$')
        plt.legend()
        ax4 = plt.subplot(3,2,4)
        fftplot(y, ax4, '$\mathcal{F}(y)$', 1/500, log = True)
        plt.legend()
plt.subplot(3,2,5)
for i in ind:
    str1 = df.iloc[i]['shock_z_axis']
    if isinstance(str1, str):
        l = str1.split(',')
        y = list(map(float,l[0:-2]))
        plt.plot(y, label = '$z$')
        plt.legend()
        ax6 = plt.subplot(3,2,6)
        fftplot(y, ax6, '$\mathcal{F}(x)$', 1/500, log = True)
        plt.legend()
plt.savefig('FFTxyz.png', dpi = 600, bbox_inches='tight',
           transparent = True)

## Flachstellenerkennung

Auf Grundlage der Frequenz einer Flachstelle werden die Fouriertransformierte und das betreffende Frequenzfenster dargestellt. Eine höhere Amplitude kann auf Flachstellen hinweisen.

Es gilt:

\begin{equation*}
	f(v) = \frac{v}{U} = \frac{v}{\pi D},
\end{equation*}
also z.B. $f(100\,\mathrm{kmh}^{-1}) \approx 10\,\mathrm{Hz}$

Das Schwingverhalten der Federstufe wird nicht modelliert.

In [ ]:
# Randomly select an index from moving vehicle
# Prefer velocities resulting in frequencies 
i = np.random.choice(df[df['speed'] > 60].index)
# Obtain wheel frequency from velocity
v = df.iloc[i]['speed']
f_flat = v/3.6/(np.pi*0.92)
# Obtain vibration data in z-direction
str1 = df.iloc[i]['shock_z_axis']
if isinstance(str1, str):
    l = str1.split(',')
    y = list(map(float,l[0:-2]))
    #plt.plot(y, label = '$z$')
    #plt.legend()
    ax = plt.subplot(1,1,1)
    fftplot(y, ax, 'FFT($z$)', 1/500, log = True)
    ax.set_xlim([0, 5*f_flat])
    (y1, y2) = ax.get_ylim()
    ax.add_patch(plt.Rectangle((f_flat, 1.1*y1), 0.2*f_flat, 0.9*(y2-y1), ec="r", fc="r"))
    plt.legend()
    plt.title('I: ' +str(i)+', v: ' + str(v) + ' km/h, f: ' + str(np.round(f_flat, 1)) + ' Hz')
plt.savefig('WheelFlat'+ str(i)+'.png', dpi = 600, bbox_inches='tight',
           transparent = True)

In [ ]:
ax = plt.subplot(1,1,1)
for k in range(10):
    #plt.figure()
    #ax = plt.subplot(1,1,1)
# Randomly select an index from moving vehicle
    i = np.random.choice(df[df['speed'] == 100].index)
    # Obtain wheel frequency from velocity
    v = df.iloc[i]['speed']
    f_flat = v/3.6/(np.pi*0.92) # Maximum wheel diameter Y25
    # Obtain vibration data in z-direction
    str1 = df.iloc[i]['shock_z_axis']
    if isinstance(str1, str):
        l = str1.split(',')
        y = list(map(float,l[0:-2]))
        #plt.plot(y, label = '$z$')
        #plt.legend()
        fftplot(y, ax, df.iloc[i]['wagonID'][2:-2], 1/500, log = True)
# Plotting and saving
ax.set_xlim([0, 5*f_flat])
ax.grid()
ax.set_ylim([1e-2, 1e2])
(y1, y2) = ax.get_ylim()
ax.add_patch(plt.Rectangle((f_flat, 1.1*y1), 0.2*f_flat, 0.9*(y2-y1), ec="r", fc="r"))
plt.title('v: ' + str(v) + ' km/h, f: ' + str(np.round(f_flat, 1)) + ' Hz')
ax.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.savefig('WheelFlat'+ str(i)+'.png', dpi = 600, bbox_inches='tight',
            transparent = True)

## Geodaten-Plots

Einige Plots, um die räumliche Verteilung zu visualisieren.

### Matplotlib

Mit der Standardbibliothek lassen sich Scatterplots erstellen, die nur auf Umwegen mit einem Kartenhintergrund versehen werden können.

In [ ]:
s = df['wagonID'].value_counts()
spart = s.index.to_list()[1:31]
for wID in spart:
    x = df[df['wagonID'] == wID]['latitude']
    y = df[df['wagonID'] == wID]['longitude']
    c = df[df['wagonID'] == wID]['shock_duration']
    plt.scatter(x=x, y=y, c = c, cmap = 'RdYlGn')

### Plotly

Die kommerzielle Bibliothek plotly bietet ein großzügiges Gratisangebot, das für die meisten Gelegenheitsanalysen ausreicht.
Sie bietet Kartenhintergründe und verschiedene Projektionen sowie eine Mapbox-Integration.

In [ ]:
s = df['wagonID'].value_counts()
df2 = df[df['wagonID'].isin(s.index.to_list()[1:101])]
data = [ dict(
        type = 'scattergeo',
        lon = df2['longitude'],
        lat = df2['latitude'],
        text = df2['x_axis'].abs(),
        mode = 'markers',
        marker_colorscale=plotly.colors.sequential.Jet,
        marker = dict(
            opacity = 0.3,
            #reversescale = True,
            #autocolorscale = False,
            #symbol = 'square',
            #colorscale = 'Jet',
            cmin = 0,
            color = df2['x_axis'].abs(),
            cmax = 0.5*df2['x_axis'].max(),
            #colorbar = dict(
            #    titleside = "right",
            #    outlinecolor = "rgba(68, 68, 68, 0)",
            #    ticks = "outside",
                #showticksuffix = "last",
                #dtick = 0.1
        #),   
        ))]

layout = dict(
        #title = 'X-Axis absolute shock values',
        geo = dict(
            scope = "europe",
            #showland = True,
            resolution = 50,
            showcoastlines=True, #coastlinecolor="RebeccaPurple",
            showland=True, #landcolor="LightGreen",
            showocean=True, oceancolor="LightBlue",
            showlakes=True, lakecolor="LightBlue",
            showrivers=True, rivercolor="LightBlue",
            countrywidth = 0.5,
            subunitwidth = 0.5,
        ),
    )


fig = dict(data=data, layout=layout )
po.plot(fig, filename='WagonShock.html')